<a href="https://githubtocolab.com/kaust-halo/geeet/blob/master/examples/notebooks/04_eepredefined_landsat_mapped_collection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

# Mapped collections 

---

In the previous notebook, we learned how to create a merged `landsat.collection` with multiple Landsat collections (7, 8, 9) and additional climate reanalysis data (ERA5). We then mapped an ET model (TSEB) to this collection. 

The `landsat.mapped_collection` is a helper function useful for mapping an arbitrary number of functions to a `landsat.collection`. 

As a motivating example, in addition to mapping the `tseb_series` model, let's also add one extra step, `extrapolate_LE`: a helper function to extrapolate instantaneous LE (W/m²) to daily ET (mm/day) using the [Jackson et al. (1983)](<https://doi.org/10.1016/0378-3774(83)90095-1>) irradiance model:

$$
ET_d \approx ET_i R
$$
where R is the ratio of instantaneous to daily radiation and can be obtained as:
$$
R =  \frac{2N}{2.45 \times 10^6 \pi \times sin(\pi \times t/N)}
$$
where N is the number of seconds between sunset and sunrise, and t is the number of seconds between sunrise and the observation time. We also apply a 1/2.45 scaling factor to convert the values to mm/day. 

In [1]:
# Use the following line to install geeet if needed:
#!pip install git+https://github.com/kaust-halo/geeet
import ee
#ee.Authenticate() # Uncomment if using Google Colab or first time using EE on this device. 
ee.Initialize()

In [2]:
import geeet
from geeet.eepredefined import landsat

region = dict(type="Point", coordinates=[38.25, 30.25])

# Define a custom workflow (TSEB model + LE extrapolation)
workflow = [
    geeet.tseb.tseb_series,
    landsat.extrapolate_LE    # this adds the "ET" band, in mm/day
]

landsat_era5_tseb_collection = landsat.mapped_collection(
    workflow,
    date_start = "2023-04-01",
    date_end = "2023-05-01",
    region = region, 
    era5 = True
)

In [3]:
landsat_era5_tseb_collection.first().bandNames().getInfo()

['SR_B1',
 'SR_B2',
 'SR_B3',
 'SR_B4',
 'SR_B5',
 'SR_B7',
 'SR_ATMOS_OPACITY',
 'SR_CLOUD_QA',
 'ST_B6',
 'ST_ATRAN',
 'ST_CDIST',
 'ST_DRAD',
 'ST_EMIS',
 'ST_EMSD',
 'ST_QA',
 'ST_TRAD',
 'ST_URAD',
 'QA_PIXEL',
 'QA_RADSAT',
 'NDVI',
 'albedo',
 'radiometric_temperature',
 'cloud_cover',
 'surface_pressure',
 'air_temperature',
 'u_component_of_wind_10m',
 'v_component_of_wind_10m',
 'surface_solar_radiation_downwards_hourly',
 'surface_thermal_radiation_downwards_hourly',
 'wind_speed',
 'solar_radiation',
 'thermal_radiation',
 'Tc',
 'Ts',
 'Tac',
 'Hc',
 'Hs',
 'LEc',
 'LEs',
 'Ra',
 'Rs',
 'Rx',
 'Ustar',
 'alphaPT',
 'iteration',
 'LE',
 'H',
 'G',
 'Rn',
 'Rns',
 'Rnc',
 'ET']

We can see that this image also contains the "ET" band.